In [12]:
from torchsummary import summary
from torch import nn
import torch

In [1]:
import sys
# sys.path.append(r'C:\AmyPython\Test1')
sys.path

['/home/wuwenjun/jupyter_code/Shannon/AlphaNet/12.30',
 '/home/wuwenjun/jupyter_code/Shannon/AlphaNet/12.30',
 '/home/ShareFolder/lgc/Modules',
 '/usr/local/miniconda/lib/python37.zip',
 '/usr/local/miniconda/lib/python3.7',
 '/usr/local/miniconda/lib/python3.7/lib-dynload',
 '',
 '/home/wuwenjun/.local/lib/python3.7/site-packages',
 '/usr/local/miniconda/lib/python3.7/site-packages',
 '/home/wuwenjun/.local/lib/python3.7/site-packages/IPython/extensions',
 '/home/wuwenjun/.ipython']

In [13]:
class AlphaNet_LSTM(nn.Module):
    def __init__(self, factor_num, fully_connect_layer_neural):
        super(AlphaNet_LSTM, self).__init__()
        self.fc1_neuron = factor_num # 108
        self.fc2_neuron = fully_connect_layer_neural # 30
        
        # Layer
        self.batch = torch.nn.BatchNorm1d(self.fc1_neuron)
        self.lstm = nn.LSTM(self.fc1_neuron,self.fc2_neuron,2,batch_first = True,bidirectional = True)
        self.batch2 = torch.nn.BatchNorm1d(self.fc2_neuron)
        self.dropout = nn.Dropout(0.3)
#         self.relu = nn.ReLU()
        self.out = nn.Linear(self.fc2_neuron, 1)

    def forward(self, x):
        x = self.batch(x)
        x = torch.transpose(x,1,2)
        _,(hn, cn) = self.lstm(x) # hn.shape: torch.Size([4, 512, 30])
        hn = hn[-1,:,:] # torch.Size([512, 30])
        hn = self.batch2(hn) # torch.Size([512, 30])
#         hn = self.relu(hn)
        hn = self.dropout(hn)
        y_pred = self.out(hn)
        return y_pred

In [18]:
alphanet = AlphaNet_LSTM(108, 30)
alphanet = alphanet.cuda()
summary(alphanet, input_size=(108, 3))

AttributeError: 'tuple' object has no attribute 'size'

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net().to(device)

summary(model, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 24, 24]             260
            Conv2d-2             [-1, 20, 8, 8]           5,020
         Dropout2d-3             [-1, 20, 8, 8]               0
            Linear-4                   [-1, 50]          16,050
            Linear-5                   [-1, 10]             510
Total params: 21,840
Trainable params: 21,840
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 0.08
Estimated Total Size (MB): 0.15
----------------------------------------------------------------
